In [1]:
import hopsworks
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
import seaborn as sns
from matplotlib import pyplot
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import joblib
import os

/u/11/zhangz13/unix/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
api = 'HeCatNGJxisb99Vf.ircWdTrkgbbZpBMU7iPN2zqDIwoTuaSX88LPeISIMJHuzP3icXixNd6JFcWUqakL'
project = hopsworks.login(api_key_value = api)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/229529
Connected. Call `.close()` to terminate connection gracefully.


In [3]:

import pandas as pd
import random
from great_expectations.dataset import PandasDataset
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

def expect(suite, column, min_val, max_val):
    suite.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column":column, 
            "min_value":min_val,
            "max_value":max_val,
        }
    )
)
suite = ExpectationSuite(expectation_suite_name="wine_dimensions")
expect(suite, "fixed_acidity", 3.5, 16.0)
expect(suite, "volatile_acidity", 0.06,1.60)
expect(suite, "citric_acid", 0.0,7.5)
expect(suite, "residual_sugar", 0.3,66.0)
expect(suite, "chlorides", 0.00,0.65)
expect(suite, "free_sulfur_dioxide", 0.8,290.0)
expect(suite, "total_sulfur_dioxide", 5.5,450.0)
expect(suite, "density", 0.95,1.03)
expect(suite, "ph", 0.3,4.5)
expect(suite, "sulphates", 0.2, 2.5)
expect(suite, "alcohol", 7.8, 15.5)
expect(suite, "quality", 0,2)


def generate_synthetic_wine(seed):
    random.seed(seed)
    return {
        "fixed_acidity": 5.0,  
        "volatile_acidity": 0.51,
        "citric_acid": 0.3,
        "residual_sugar":1.0,
        "chlorides":random.uniform(0.55, 0.56),
        "free_sulfur_dioxide":30.0,
        "total_sulfur_dioxide":32.0,
        "density":0.98,
        "ph":3.6,
        "sulphates":random.uniform(1.3, 1.5),
        "alcohol":11.5,
        "quality":1
    }

wine_fg = fs.get_or_create_feature_group(
    name="wine_2",
    version=6,
    primary_key=["fixed_acidity","volatile_acidity","citric_acid", "residual_sugar"	,"chlorides",	
                 "free_sulfur_dioxide",	"total_sulfur_dioxide",	"density","pH","sulphates","alcohol","quality"], 
    description="For new wine data")

num_samples = 20  # Number of synthetic samples to generate

synthetic_wines_new1 = [generate_synthetic_wine(seed=i) for i in range(num_samples)]
# synthetic_wines_new = generate_synthetic_wine(seed=2)
# Convert the list of dictionaries to a DataFrame
synthetic_wines_df = pd.DataFrame(synthetic_wines_new1)
synthetic_wines_ge_df = PandasDataset(synthetic_wines_df)
# Validate the entire DataFrame
results = synthetic_wines_ge_df.validate(expectation_suite=suite, result_format="SUMMARY")

# Check if the new data meets the expectations
if results["success"]:
    wine_fg.insert(synthetic_wines_df, overwrite=False, operation="append")
    print("All synthetic wine data inserted successfully.")
else:
    print("Data validation failed:", results)


Uploading Dataframe: 100.00% |██████████| Rows 20/20 | Elapsed Time: 00:06 | Remaining Time: 00:00


Launching job: wine_2_6_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/229529/jobs/named/wine_2_6_offline_fg_materialization/executions
All synthetic wine data inserted successfully.


In [4]:
synthetic_wines_ge_df

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol,quality
0,5.0,0.51,0.3,1.0,0.558444,30.0,32.0,0.98,3.6,1.451591,11.5,1
1,5.0,0.51,0.3,1.0,0.551344,30.0,32.0,0.98,3.6,1.469487,11.5,1
2,5.0,0.51,0.3,1.0,0.559560,30.0,32.0,0.98,3.6,1.489565,11.5,1
3,5.0,0.51,0.3,1.0,0.552380,30.0,32.0,0.98,3.6,1.408846,11.5,1
4,5.0,0.51,0.3,1.0,0.552360,30.0,32.0,0.98,3.6,1.320633,11.5,1
5,5.0,0.51,0.3,1.0,0.556229,30.0,32.0,0.98,3.6,1.448357,11.5,1
6,5.0,0.51,0.3,1.0,0.557933,30.0,32.0,0.98,3.6,1.464391,11.5,1
7,5.0,0.51,0.3,1.0,0.553238,30.0,32.0,0.98,3.6,1.330170,11.5,1
8,5.0,0.51,0.3,1.0,0.552267,30.0,32.0,0.98,3.6,1.492459,11.5,1
9,5.0,0.51,0.3,1.0,0.554630,30.0,32.0,0.98,3.6,1.374662,11.5,1


In [ ]:
import os
import modal
    
LOCAL=False

if LOCAL == False:
   stub = modal.Stub()
   hopsworks_image = modal.Image.debian_slim().pip_install(["hopsworks","joblib","seaborn","scikit-learn==1.1.1","dataframe-image"])
   @stub.function(image=hopsworks_image, schedule=modal.Period(days=1), secret=modal.Secret.from_name("id2223"))
   def f():
       g()

def g():
    import pandas as pd
    import hopsworks
    import joblib
    import datetime
    from PIL import Image
    from datetime import datetime
    import dataframe_image as dfi
    from sklearn.metrics import confusion_matrix
    from matplotlib import pyplot
    import seaborn as sns
    import requests
    import hopsworks
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import classification_report
    from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
    import seaborn as sns
    from matplotlib import pyplot
    from hsml.schema import Schema
    from hsml.model_schema import ModelSchema
    import joblib
    model_version = 1
    api = 'HeCatNGJxisb99Vf.ircWdTrkgbbZpBMU7iPN2zqDIwoTuaSX88LPeISIMJHuzP3icXixNd6JFcWUqakL'
    project = hopsworks.login(api_key_value = api)
    fs = project.get_feature_store()
    
    mr = project.get_model_registry()
    model = mr.get_model("wine_model_2", version=model_version)
    model_dir = model.download()
    print("model_dir:",model_dir)
    model = joblib.load(model_dir + "/wine_model_2.pkl")
    
    feature_view = fs.get_feature_view(name="wine_2", version=6)
    batch_data = feature_view.get_batch_data()
    
    y_pred = model.predict(batch_data)
    print("Wine Quality Predictions: ", y_pred)
    print("Wine Quality average:: ", y_pred.mean())


if __name__ == "__main__":
    if LOCAL == True :
        g()
    else:
        with stub.run():
            f.remote()


